In [4]:
# import libraries
import json
import pandas as pd

In [6]:
with open("YOUR_JSON_FILE", "r", encoding="utf-8") as f:
    data1 = json.load(f)
df1 = pd.DataFrame(data1)

In [7]:
alldata = pd.concat([df1], ignore_index=True) # add all files

In [9]:
alldata = alldata.drop(["platform", "ip_addr", "offline_timestamp", "incognito_mode"], axis=1) # drop irrelevant columns for simplicity

In [11]:
alldata["ts"] = pd.to_datetime(alldata["ts"]) # transform time stamps to date format

In [13]:
alldata["minutes_played"] = alldata["ms_played"] / 60000 # transform milliseconds to minutes

In [14]:
df_podcasts = alldata[alldata["spotify_track_uri"].isnull()] # separate dataframe for podcasts

In [15]:
alldata = alldata[~alldata["spotify_track_uri"].isnull()] # remove everything that are not tracks from the main dataframe

In [16]:
alldata = alldata[alldata["minutes_played"] > 0.2] # remove streaming entries that were listened to for less than 0.2 minutes

In [18]:
# Find the first stream date for each song
first_played = alldata.groupby('spotify_track_uri')['ts'].min().reset_index(name='first_play') # group songs and use the earliest time stamp
first_played['date'] = first_played['first_play'].dt.date # set the date
first_played

,spotify_track_uri,first_play,date
0,spotify:track:000xQL6tZNLJzIrtIgxqSl,2017-04-01 13:52:14+00:00,2017-04-01
1,spotify:track:003eoIwxETJujVWmNFMoZy,2018-06-16 11:38:15+00:00,2018-06-16
2,spotify:track:003vvx7Niy0yvhvHt4a68B,2020-10-10 13:15:08+00:00,2020-10-10
3,spotify:track:0066u0Nc8LLFYvln3BmBJd,2018-01-25 06:21:11+00:00,2018-01-25
4,spotify:track:007PPvZtGDYHSEhYPxqIfC,2019-01-19 15:14:40+00:00,2019-01-19
...,...,...,...
19430,spotify:track:7zolcuA2RVIATBjrAs0VRD,2019-03-17 21:15:22+00:00,2019-03-17
19431,spotify:track:7zv8Fey2BaVJe0PomkkA16,2017-04-11 15:59:49+00:00,2017-04-11
19432,spotify:track:7zwn1eykZtZ5LODrf7c0tS,2024-11-12 10:36:44+00:00,2024-11-12
19433,spotify:track:7zybSU9tFO9HNlwmGF7stc,2024-08-23 20:08:12+00:00,2024-08-23


In [20]:
alldata = alldata.merge(first_played, on='spotify_track_uri', how='left') # merge first_played data with the main dataframe
alldata

,ts,ms_played,conn_country,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,minutes_played,first_play,date
0,2024-05-31 10:01:48+00:00,147629,SE,Bad Memories,MEDUZA,MEDUZA,spotify:track:3JhgPs7rbmzLBWRvHlel8U,None,None,None,trackdone,trackdone,True,False,False,2.460483,2023-10-21 23:27:16+00:00,2023-10-21
1,2024-05-31 10:47:03+00:00,190367,SE,The Anchor,Bastille,Wild World,spotify:track:0qwpBiu4uhW9hiQ6P4NePz,None,None,None,trackdone,logout,True,False,False,3.172783,2020-12-24 15:24:30+00:00,2020-12-24
2,2024-05-31 14:11:59+00:00,209596,SE,Sorry To Me Too,Julia Michaels,Sorry To Me Too,spotify:track:4SDccIA3pVkjaZsivDPaV5,None,None,None,fwdbtn,trackdone,True,False,False,3.493267,2022-12-22 21:59:30+00:00,2022-12-22
3,2024-05-31 14:14:10+00:00,129986,SE,Samma gamla vanliga,A36,AREA 36,spotify:track:4wGFTaengFG4o6nYZAK1Xg,None,None,None,trackdone,trackdone,True,False,False,2.166433,2022-10-08 09:39:57+00:00,2022-10-08
4,2024-05-31 14:16:39+00:00,147986,SE,Forget You,FAST BOY,Forget You,spotify:track:1YWr18dWRmuvPgAsPBsOow,None,None,None,trackdone,trackdone,True,False,False,2.466433,2023-03-08 15:52:31+00:00,2023-03-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126324,2017-10-24 14:11:59+00:00,228306,DE,Young - Midnight Kids Remix,The Chainsmokers,Young,spotify:track:2g1dcVwh2fbCKzbxX2RgwZ,None,None,None,trackdone,trackdone,True,False,False,3.805100,2017-10-24 14:11:59+00:00,2017-10-24
126325,2017-10-24 14:15:57+00:00,200727,DE,Bad At Love - Dillon Francis Remix,Halsey,Bad At Love Remixes,spotify:track:5N2wcrtSauxBz5tHQI0Wby,None,None,None,trackdone,trackdone,True,False,False,3.345450,2017-10-24 14:15:57+00:00,2017-10-24
126326,2017-10-24 14:18:39+00:00,161322,DE,Be Mine,Ofenbach,Be Mine,spotify:track:2KklXplRtxMsBYo474Es0w,None,None,None,trackdone,trackdone,True,False,False,2.688700,2017-09-02 15:41:13+00:00,2017-09-02
126327,2017-10-24 14:22:11+00:00,211896,DE,Know Love,DVBBS,Blood Of My Blood,spotify:track:1YLSTCHPgTtXjG06sljS85,None,None,None,trackdone,trackdone,True,False,False,3.531600,2017-10-24 14:22:11+00:00,2017-10-24


In [23]:
# make separate dataframes if a yearly wrapped is wanted
df_2024 = alldata[alldata["ts"].dt.year == 2024]
df_2023 = alldata[alldata["ts"].dt.year == 2023]
df_2022 = alldata[alldata["ts"].dt.year == 2022]
df_2021 = alldata[alldata["ts"].dt.year == 2021]
df_2020 = alldata[alldata["ts"].dt.year == 2020]
df_2019 = alldata[alldata["ts"].dt.year == 2019]
df_2018 = alldata[alldata["ts"].dt.year == 2018]
df_2017 = alldata[alldata["ts"].dt.year == 2017]
df_2016 = alldata[alldata["ts"].dt.year == 2016]
df_2015 = alldata[alldata["ts"].dt.year == 2015]

In [25]:
df_2024

,ts,ms_played,conn_country,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,minutes_played,first_play,date
0,2024-05-31 10:01:48+00:00,147629,SE,Bad Memories,MEDUZA,MEDUZA,spotify:track:3JhgPs7rbmzLBWRvHlel8U,None,None,None,trackdone,trackdone,True,False,False,2.460483,2023-10-21 23:27:16+00:00,2023-10-21
1,2024-05-31 10:47:03+00:00,190367,SE,The Anchor,Bastille,Wild World,spotify:track:0qwpBiu4uhW9hiQ6P4NePz,None,None,None,trackdone,logout,True,False,False,3.172783,2020-12-24 15:24:30+00:00,2020-12-24
2,2024-05-31 14:11:59+00:00,209596,SE,Sorry To Me Too,Julia Michaels,Sorry To Me Too,spotify:track:4SDccIA3pVkjaZsivDPaV5,None,None,None,fwdbtn,trackdone,True,False,False,3.493267,2022-12-22 21:59:30+00:00,2022-12-22
3,2024-05-31 14:14:10+00:00,129986,SE,Samma gamla vanliga,A36,AREA 36,spotify:track:4wGFTaengFG4o6nYZAK1Xg,None,None,None,trackdone,trackdone,True,False,False,2.166433,2022-10-08 09:39:57+00:00,2022-10-08
4,2024-05-31 14:16:39+00:00,147986,SE,Forget You,FAST BOY,Forget You,spotify:track:1YWr18dWRmuvPgAsPBsOow,None,None,None,trackdone,trackdone,True,False,False,2.466433,2023-03-08 15:52:31+00:00,2023-03-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25698,2024-05-31 09:41:49+00:00,191000,SE,Shots - Broiler Remix,Imagine Dragons,Smoke + Mirrors,spotify:track:0fsEFwLSJTSHSawhe864Cp,None,None,None,trackdone,trackdone,True,False,False,3.183333,2023-02-22 14:46:05+00:00,2023-02-22
25699,2024-05-31 09:45:38+00:00,229173,SE,Atlantis,Seafret,Tell Me It's Real,spotify:track:1Fid2jjqsHViMX6xNH70hE,None,None,None,trackdone,trackdone,True,False,False,3.819550,2022-12-01 21:45:09+00:00,2022-12-01
25700,2024-05-31 09:51:07+00:00,272547,SE,No One’s Here To Sleep,Naughty Boy,Hotel Cabana,spotify:track:0txjdg6j38GxiWzfH2Skpg,None,None,None,trackdone,trackdone,True,False,False,4.542450,2020-12-24 15:55:40+00:00,2020-12-24
25701,2024-05-31 09:53:47+00:00,159446,SE,Good Thing,Suriel Hess,Good Thing,spotify:track:2RUwpvZSfNTVi8JzuOuhQc,None,None,None,trackdone,trackdone,True,False,False,2.657433,2023-03-13 17:44:12+00:00,2023-03-13


In [26]:
# add the total minutes up for alltime and every year
total_minutes = alldata["minutes_played"].sum()
minutes_2024 = df_2024["minutes_played"].sum()
minutes_2023 = df_2023["minutes_played"].sum()
minutes_2022 = df_2022["minutes_played"].sum()
minutes_2021 = df_2021["minutes_played"].sum()
minutes_2020 = df_2020["minutes_played"].sum()
minutes_2019 = df_2019["minutes_played"].sum()
minutes_2018 = df_2018["minutes_played"].sum()
minutes_2017 = df_2017["minutes_played"].sum()
minutes_2016 = df_2016["minutes_played"].sum()
minutes_2015 = df_2015["minutes_played"].sum()
print(f"Total minutes played: {total_minutes:.0f}")
print(f"2024 minutes played: {minutes_2024:.0f}")
print(f"2023 minutes played: {minutes_2023:.0f}")
print(f"2022 minutes played: {minutes_2022:.0f}")
print(f"2021 minutes played: {minutes_2021:.0f}")
print(f"2020 minutes played: {minutes_2020:.0f}")
print(f"2019 minutes played: {minutes_2019:.0f}")
print(f"2018 minutes played: {minutes_2018:.0f}")
print(f"2017 minutes played: {minutes_2017:.0f}")
print(f"2016 minutes played: {minutes_2016:.0f}")
print(f"2015 minutes played: {minutes_2015:.0f}")

Total minutes played: 368099
2024 minutes played: 46726
2023 minutes played: 43657
2022 minutes played: 30070
2021 minutes played: 31295
2020 minutes played: 39353
2019 minutes played: 69528
2018 minutes played: 56136
2017 minutes played: 26239
2016 minutes played: 12815
2015 minutes played: 12279


In [27]:
unique_track_names = df_2024["master_metadata_track_name"].nunique() # check how many unique tracks were listened to in 2024
print(f"Number of unique tracks: {unique_track_names}")

Number of unique tracks: 4056


In [28]:
# get stats for tracks that include track name, artist, total minutes played, and times played
def get_track_stats(df):
    # Group by track name
    track_stats = (
        df.groupby("spotify_track_uri")
        .agg(
            track = ('master_metadata_track_name', 'first'),
            artist = ('master_metadata_album_artist_name', 'first'),
            minutes_played=("minutes_played", "sum"),# Total minutes played
            play_count=("minutes_played", "count") # Number of times played
        )
        .reset_index()
    )
    track_stats = track_stats.sort_values(by="minutes_played", ascending=False) # sort values by most minutes played
    return track_stats


In [29]:
track_stats = get_track_stats(alldata) # get stats for all songs ever played
track_stats.head(20) # display the top 20 most played songs of all time

,spotify_track_uri,track,artist,minutes_played,play_count
14485,spotify:track:5nFyaoBWwwMJ9fv33tZ2MB,What I Need (feat. Kehlani),Hayley Kiyoko,658.335300,188
957,spotify:track:0Mf6WhWLfOpzOxacvbcboj,rock + roll,EDEN,551.369217,124
10248,spotify:track:44nX7Y9AgkHyinw8RSXOgP,Maybe I Am,FLETCHER,450.148667,174
139,spotify:track:03PM8jkLPwI6cDcZBvwCSL,He'll Never Love You (HNLY),Hayley Kiyoko,395.770867,107
8553,spotify:track:3OXpdrHLT4IqM76RajgM6Q,drugs,EDEN,395.112800,80
14245,spotify:track:5hqxBvQJ3XJDSbxT9vyyqA,Alles neu,Peter Fox,392.754133,107
10789,spotify:track:4Ivo07fh9EyPtPinrxXlC4,"Violin Concerto in E Minor, Op. 64, MWV O 14: ...",Felix Mendelssohn,375.663183,40
18309,spotify:track:7LVHVU3tWfcxj5aiPFEW4Q,Fix You,Coldplay,350.510283,83
4423,spotify:track:1k7jAhbUihs1zx3rbhWgj0,Doing Better,FLETCHER,342.166233,152
7722,spotify:track:340UVheS8z3ncW9TTUhAbc,Nightmare,Halsey,334.067583,90


In [30]:
track_stats_2024 = get_track_stats(df_2024) # get stats for all songs played in 2024
track_stats_2024.head(20) # display the top 20 most played songs of all time

,spotify_track_uri,track,artist,minutes_played,play_count
2313,spotify:track:44nX7Y9AgkHyinw8RSXOgP,Maybe I Am,FLETCHER,450.148667,174
988,spotify:track:1k7jAhbUihs1zx3rbhWgj0,Doing Better,FLETCHER,342.166233,152
1175,spotify:track:2262bWmqomIaJXwCRHr13j,Sailor Song,Gigi Perez,264.437317,72
1563,spotify:track:2iv9L7pBxfuGIRbJx7ip1D,Make Me Cry (feat. Deadbeat Girl),Ari Abdul,263.972417,78
4173,spotify:track:7dIi6DS9RA6PDTU90CcNoV,fuß baumeln,Zartmann,239.218917,99
3903,spotify:track:6vc2Jq2vaGu8z326kSrw92,Your Love Is My Drug,Kesha,233.337433,73
1630,spotify:track:2prqm9sPLj10B4Wg0wE5x9,BLUE,Billie Eilish,229.378633,46
210,spotify:track:0Mf6WhWLfOpzOxacvbcboj,rock + roll,EDEN,225.236867,52
3844,spotify:track:6p9vTtbwiVHMahR5UwNI7k,Ungrateful Eyes,Jon Bellion,215.194767,70
1878,spotify:track:3K7vPyMCcecKRotnu08MMP,Crazier Things (with Noah Kahan),Chelsea Cutler,205.961417,50


In [36]:
unique_artist_names = df_2024["master_metadata_album_artist_name"].nunique() # get all unique artists in 2024
print(f"Number of different artists: {unique_artist_names}") 

Number of different artists: 1647


In [38]:
# get stats of artists: artist name and total minutes played
def get_artist_stats(df):
    artist_stats = (
        df.groupby("master_metadata_album_artist_name")
        .agg(
            minutes_played=("minutes_played", "sum"),
        )
        .reset_index()
    )
    artist_stats = artist_stats.sort_values(by="minutes_played", ascending=False)
    artist_stats.reset_index(drop=True, inplace=True)
    return artist_stats

In [40]:
artist_stats = get_artist_stats(alldata) # get stats for all artists ever played
artist_stats.head(10) # get top 10 artists of all time

,master_metadata_album_artist_name,minutes_played
0,FLETCHER,7861.672200
1,Justin Bieber,6646.529767
2,Lauv,6182.905117
3,One Direction,6133.704083
4,Halsey,5120.093633
5,Panic! At The Disco,4209.309667
6,Tate McRae,4088.370900
7,Taylor Swift,3888.212683
8,Hayley Kiyoko,3791.469250
9,Troye Sivan,3704.016667


In [42]:
artist_stats_2024 = get_artist_stats(df_2024) # get stats for the artists played in 2024
artist_stats_2024.head(10) # get top 10 artists of 2024

,master_metadata_album_artist_name,minutes_played
0,FLETCHER,4385.639117
1,girl in red,2028.132500
2,Billie Eilish,1349.633650
3,Gracie Abrams,1321.710017
4,Chelsea Cutler,1013.054433
5,Peter Fox,993.218983
6,LUNA,953.034217
7,EDEN,797.682467
8,Cornelia Funke,768.511217
9,Charli xcx,664.681200


In [44]:
artist_stats_2023 = get_artist_stats(df_2023) # get stats for the artists played in 2023
artist_stats_2023.head(10) # get top 10 artists of 2023

,master_metadata_album_artist_name,minutes_played
0,Tate McRae,1795.408767
1,FLETCHER,1502.586750
2,Irina Kairatovna,918.633617
3,Bastille,812.249067
4,The Chainsmokers,722.842333
5,YUNGBLUD,543.506300
6,Lauv,526.944283
7,Avicii,463.389733
8,Kygo,446.069083
9,shy martin,428.536567


In [45]:
artist_stats_2022 = get_artist_stats(df_2022) # get stats for the artists played in 2022
artist_stats_2022.head(10) # get top 10 artists of 2022

,master_metadata_album_artist_name,minutes_played
0,Tate McRae,1306.092767
1,Peter Fox,793.813267
2,JP Saxe,784.003567
3,Taylor Swift,761.982167
4,Sasha Alex Sloan,704.134483
5,Avicii,650.957550
6,Khalid,524.885450
7,Lorde,503.006100
8,ATEEZ,501.075950
9,Alec Benjamin,461.056250


In [50]:
artist_stats_2021 = get_artist_stats(df_2021) # get stats for the artists played in 2021
artist_stats_2021.head(10) # get top 10 artists of 2021

,master_metadata_album_artist_name,minutes_played
0,JP Saxe,909.213400
1,Khalid,845.821500
2,mgk,833.472600
3,BLACKPINK,631.593483
4,Lauv,608.724683
5,Lil Nas X,595.421367
6,Halsey,541.644967
7,BTS,503.659250
8,Charlotte Lawrence,473.663983
9,jxdn,459.755350


In [52]:
favorite_artist = artist_stats_2024['master_metadata_album_artist_name'][0] # get favorite artist in 2024

In [54]:
# Filter data for the favorite artist
artist_data = df_2024[df_2024['master_metadata_album_artist_name'] == favorite_artist] 

# Extract date (ignoring time) and group by date
artist_data['date'] = artist_data['ts'].dt.date
days_listened = artist_data['date'].nunique()

# Display the result
print(f"You listened to {favorite_artist} on {days_listened} unique days in the year.")

You listened to FLETCHER on 174 unique days in the year.


C:\Users\leham\AppData\Local\Temp\ipykernel_39776\4011521509.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artist_data['date'] = artist_data['ts'].dt.date


In [55]:
unique_album_names = df_2024["master_metadata_album_album_name"].nunique() # get unique albums of 2024
print(f"Number of different albums: {unique_album_names}")

Number of different albums: 3026


In [58]:
# get album stats: album name, artist, minutes played
def get_album_stats(df): 
    album_stats = (
        df.groupby(["master_metadata_album_album_name", "master_metadata_album_artist_name"])
        .agg(
            minutes_played=("minutes_played", "sum"),# Total minutes played
            artist = ('master_metadata_album_artist_name', 'first')
        )
        .reset_index()
    )
    album_stats = album_stats.sort_values(by="minutes_played", ascending=False) # sort by minutes played
    return album_stats

In [60]:
album_stats = get_album_stats(alldata) # get album stats for alltime
album_stats.head(10) # get top 10 albums of all time

,master_metadata_album_album_name,master_metadata_album_artist_name,minutes_played,artist
3030,Expectations,Hayley Kiyoko,2700.860550,Hayley Kiyoko
4970,In Search Of The Antidote,FLETCHER,2291.858817,FLETCHER
4749,I met you when I was 18.,Lauv,2229.287033,Lauv
12186,~how i'm feeling~,Lauv,2185.502433,Lauv
11750,i think you think too much of me,EDEN,2118.221317,EDEN
3462,Free Spirit,Khalid,2068.969200,Khalid
8992,Stadtaffe,Peter Fox,2030.395667,Peter Fox
1091,Believe,Justin Bieber,1778.988300,Justin Bieber
1619,Camila,Camila Cabello,1587.015350,Camila Cabello
10488,Up All Night,One Direction,1581.258200,One Direction


In [64]:
album_stats_2024 = get_album_stats(df_2024) # get album stats for 2024
album_stats_2024.head(10) # get top 10 albums of 2024

,master_metadata_album_album_name,master_metadata_album_artist_name,minutes_played,artist
1232,In Search Of The Antidote,FLETCHER,2291.858817,FLETCHER
981,HIT ME HARD AND SOFT,Billie Eilish,1189.512733,Billie Eilish
1174,I'M DOING IT AGAIN BABY!,girl in red,876.974767,girl in red
611,Die Wilden Hühner und die Liebe,Cornelia Funke,768.511217,Cornelia Funke
2452,The Secret of Us,Gracie Abrams,707.286217,Gracie Abrams
2217,Stellaria,Chelsea Cutler,626.127483,Chelsea Cutler
2939,if i could make it go quiet,girl in red,614.458000,girl in red
2931,i think you think too much of me,EDEN,604.841800,EDEN
3069,verlierer,LUNA,588.571033,LUNA
912,Girl Of My Dreams - Deluxe,FLETCHER,418.302500,FLETCHER


In [68]:
skipped_songs = df_2024[df_2024['skipped'] == True] # dataframe with all skipped streams

skipped_counts = skipped_songs.groupby('master_metadata_track_name').size() # count how many times each song is skkipped

skipped_counts = skipped_counts.sort_values(ascending=False) # sort songs by skipps

# Display the top 10 most skipped songs
print(skipped_counts.head(10)) 

master_metadata_track_name
Doing Better           36
Antidote               23
Up At Night            22
Ungrateful Eyes        21
Turn Red               20
Sailor Song            19
Officially Mine        19
Before You Leave Me    18
fuß baumeln            18
Hot Killer             18
dtype: int64


In [70]:
df_2024['hour'] = df_2024['ts'].dt.hour # get the hours from the time stamp

# Define time intervals
def time_of_day(hour):
    if 6 <= hour < 10:
        return 'Morning'  # 5 AM to 9:59 AM
    elif 10 <= hour < 14:
        return 'Midday'  # 10 AM to 1:59 PM
    elif 14 <= hour < 18:
        return 'Afternoon'  # 2 PM to 5:59 PM
    elif 18 <= hour < 23:
        return 'Evening'    # 6 PM to 10:59 PM
    else:
        return 'Night'  # 11 PM to 5:59 AM

# Apply the function to create a new column for the time of day
df_2024['time_of_day'] = df_2024['hour'].apply(time_of_day)

# Group by track name and time of day, then count the occurrences
song_time_stats = df_2024.groupby(['spotify_track_uri', 'time_of_day', 'master_metadata_track_name'])['minutes_played'].sum().reset_index(name='total minutes')
# groupby(['conn_country', 'master_metadata_track_name'])['minutes_played'].sum().reset_index(name='total_minutes')
song_time_stats = song_time_stats.sort_values(by=['time_of_day', 'total minutes'], ascending=[True, False])
top_3_per_time_of_day = song_time_stats.groupby('time_of_day').head(3)
top_3_per_time_of_day

C:\Users\leham\AppData\Local\Temp\ipykernel_39776\1246811211.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2024['hour'] = df_2024['ts'].dt.hour
C:\Users\leham\AppData\Local\Temp\ipykernel_39776\1246811211.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2024['time_of_day'] = df_2024['hour'].apply(time_of_day)


,spotify_track_uri,time_of_day,master_metadata_track_name,total minutes
3895,spotify:track:44nX7Y9AgkHyinw8RSXOgP,Afternoon,Maybe I Am,112.417400
1970,spotify:track:2262bWmqomIaJXwCRHr13j,Afternoon,Sailor Song,102.520850
1666,spotify:track:1k7jAhbUihs1zx3rbhWgj0,Afternoon,Doing Better,90.906233
2638,spotify:track:2iv9L7pBxfuGIRbJx7ip1D,Evening,Make Me Cry (feat. Deadbeat Girl),59.094283
6636,spotify:track:6vc2Jq2vaGu8z326kSrw92,Evening,Your Love Is My Drug,52.661117
7071,spotify:track:7dIi6DS9RA6PDTU90CcNoV,Evening,fuß baumeln,48.741233
3897,spotify:track:44nX7Y9AgkHyinw8RSXOgP,Midday,Maybe I Am,110.363500
3170,spotify:track:3K7vPyMCcecKRotnu08MMP,Midday,Crazier Things (with Noah Kahan),110.243350
3943,spotify:track:46sYl0o21xmXFK1a3y94Ha,Midday,When She Smiles,102.101900
3898,spotify:track:44nX7Y9AgkHyinw8RSXOgP,Morning,Maybe I Am,143.243550


In [72]:
df_2024['date'] = df_2024['ts'].dt.date #  extract date
unique_days = df_2024['date'].nunique()

print(f"You listened to Spotify on {unique_days} days in 2024.")

You listened to Spotify on 324 days in 2024.


C:\Users\leham\AppData\Local\Temp\ipykernel_39776\3985077691.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2024['date'] = df_2024['ts'].dt.date


In [73]:
unique_countries = df_2024['conn_country'].nunique() # get how many unique countries was played from in 2024
print(f"You listened to Spotify in {unique_countries} different countries in 2024.")

You listened to Spotify in 10 different countries in 2024.


In [75]:
# Group by country and track name, then sum the minutes played
song_country_stats = df_2024.groupby(['conn_country', 'master_metadata_track_name'])['minutes_played'].sum().reset_index(name='total_minutes')

# Sort by country and total minutes (in descending order)
song_country_stats = song_country_stats.sort_values(by=['conn_country', 'total_minutes'], ascending=[True, False])

# For each country, get the top song
top_song_per_country = song_country_stats.groupby('conn_country').head(3)

# Display the result
print(top_song_per_country)

     conn_country                master_metadata_track_name  total_minutes
13             AT                             Body And Mind      18.667200
31             AT                        I'll Call You Mine      10.097250
12             AT              Bitter (feat. Trevor Daniel)       9.737450
152            CZ                               Up At Night      21.649017
160            CZ                                       Zzz      11.571950
164            CZ                               rock + roll       9.776000
527            DE                                  немерено      52.821000
430            DE                            That’s So True      39.795050
195            DE                                      BLUE      22.837767
545            DK                                The Anchor       3.413650
528            DK                                 ANNA BOOK       3.210783
537            DK  In A Perfect World (with Julia Michaels)       3.183550
559            FI        

In [77]:
# import spotipy to use API
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [82]:
client_id = "YOUR_CLIENT_ID"
client_secret = "YOUR_CLIENT_SECRET"
redirect_uri = "YOUR_URI"

In [85]:
# athenticate
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_uri,
                                               scope=["user-library-read", "playlist-read-private"],
                                               open_browser=False))

def get_auth_response_manual():
    # Generate URL for authentication
    auth_url = sp.auth_manager.get_authorize_url()
    print(f"Please go to the following URL and authorize: {auth_url}")
    response = input("Paste the URL you were redirected to: ")
    code = sp.auth_manager.parse_response_code(response)
    sp.auth_manager.get_access_token(code)


In [88]:
# get the genre belonging to ech artist
def get_artist_genre(artist_name):
    # Search for the artist on Spotify
    result = sp.search(q=artist_name, type='artist', limit=1)
    
    if result['artists']['items']:
        # Get the artist's Spotify ID
        artist_id = result['artists']['items'][0]['id']
        # Get the artist's details, including genres
        artist_info = sp.artist(artist_id)
        return artist_info['genres']  # Returns a list of genres
    else:
        return []  # If no genres found, return empty list

In [90]:
# get genre of exempel artist
genre = get_artist_genre('Lorna Shore')

In [92]:
genre

['blackened deathcore', 'melodic metalcore', 'symphonic deathcore']

## Monthly recap

In [94]:
# Extract the year and month from the timestamp and create a new column
df_2024['year_month'] = df_2024['ts'].dt.to_period('M')

# Group by year_month and track name and calculate total minutes played
top_songs_per_month_df = df_2024.groupby(['year_month', 'master_metadata_track_name']).agg(
    total_minutes=('minutes_played', 'sum')
).reset_index()

# Sort by year_month and total_minutes
top_songs_per_month = top_songs_per_month_df.sort_values(by=['year_month', 'total_minutes'], ascending=[True, False])

# To get the top song for each month:
top_songs_per_month = top_songs_per_month.groupby('year_month').head(3)

# Display the top song for each month
print(top_songs_per_month)


     year_month                   master_metadata_track_name  total_minutes
428     2024-01                                     немерено     171.239967
344     2024-01                                    arschloch      93.645983
42      2024-01                                Body And Mind      91.115033
755     2024-02                                     Too Much     110.297283
503     2024-02  Come Marry Me (feat. Pete Fox) - Radio Edit      93.855867
846     2024-02                                         müde      75.330767
1007    2024-03                                 Doing Better     228.434917
1392    2024-03                              Ungrateful Eyes     181.948483
876     2024-03                                 100 Bad Days     156.891733
1859    2024-04                                  fuß baumeln     172.793600
1579    2024-04                                 Doing Better     128.921433
1704    2024-04                                   Maybe I Am     107.336517
2417    2024

C:\Users\leham\AppData\Local\Temp\ipykernel_39776\1127004581.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_2024['year_month'] = df_2024['ts'].dt.to_period('M')
C:\Users\leham\AppData\Local\Temp\ipykernel_39776\1127004581.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2024['year_month'] = df_2024['ts'].dt.to_period('M')


In [96]:
# Group by year_month and track name and calculate total minutes played
top_albums_per_month = df_2024.groupby(['year_month', 'master_metadata_album_album_name', 'master_metadata_album_artist_name']).agg(
    total_minutes=('minutes_played', 'sum'),
).reset_index()

# Sort by year_month and total_minutes
top_albums_per_month = top_albums_per_month.sort_values(by=['year_month', 'total_minutes'], ascending=[True, False])

# To get the top song for each month:
top_albums_per_month = top_albums_per_month.groupby('year_month').head(1)

# Display the top song for each month
print(top_albums_per_month)

     year_month  master_metadata_album_album_name  \
306     2024-01                         verlierer   
658     2024-02                         verlierer   
906     2024-03         In Search Of The Antidote   
1365    2024-04         In Search Of The Antidote   
1683    2024-05   Die Wilden Hühner und die Liebe   
2326    2024-06              HIT ME HARD AND SOFT   
3011    2024-07              HIT ME HARD AND SOFT   
3752    2024-08  i think you think too much of me   
4096    2024-09                  The Secret of Us   
4573    2024-10       if i could make it go quiet   
4934    2024-11                         Stellaria   
5146    2024-12                         Stellaria   

     master_metadata_album_artist_name  total_minutes  
306                               LUNA     263.439150  
658                               LUNA     244.570983  
906                           FLETCHER     648.730500  
1365                          FLETCHER     634.221267  
1683                    Cornel

In [98]:
# make separate dataframes for each month
df_2024_01 = df_2024[df_2024["year_month"].dt.month== 1]
df_2024_02 = df_2024[df_2024["year_month"].dt.month== 2]
df_2024_03 = df_2024[df_2024["year_month"].dt.month== 3]
df_2024_04 = df_2024[df_2024["year_month"].dt.month== 4]
df_2024_05 = df_2024[df_2024["year_month"].dt.month== 5]
df_2024_06 = df_2024[df_2024["year_month"].dt.month== 6]
df_2024_07 = df_2024[df_2024["year_month"].dt.month== 7]
df_2024_08 = df_2024[df_2024["year_month"].dt.month== 8]
df_2024_09 = df_2024[df_2024["year_month"].dt.month== 9]
df_2024_10 = df_2024[df_2024["year_month"].dt.month== 10]
df_2024_11 = df_2024[df_2024["year_month"].dt.month== 11]
df_2024_12 = df_2024[df_2024["year_month"].dt.month== 12]

In [100]:
# get songs that were newly discovered in a specific month
def get_new_discoveries(df, year, month):
    songs = df[
        (df['first_play'].dt.year == year) &
        (df['first_play'].dt.month == month)
    ]
    disc = songs.groupby(['spotify_track_uri']).agg(
        total_minutes=('minutes_played', 'sum'),
        track = ('master_metadata_track_name', 'first'),
        artist = ('master_metadata_album_artist_name', 'first')
    ).reset_index()
    disc = disc.sort_values(by=['total_minutes'], ascending=[False])
    return disc

In [98]:
disc_01 = get_new_discoveries(df_2024_01, 2024, 1)
disc_01

,spotify_track_uri,total_minutes,track,artist
127,spotify:track:6KRiMyT78RNeatgjnI7sPM,171.239967,немерено,лампабикт
6,spotify:track:0DNAisWcJwei0pu3t8TyMM,93.645983,arschloch,LUNA
16,spotify:track:0rFAE6PNRKkw4b6nxCK7b1,69.791450,THRILLER (with Clara Mae),DREAMDNVR
41,spotify:track:1uAOCTevGnyKIDbgZdOCnE,62.903867,Voilà,Barbara Pravi
143,spotify:track:705PQfRlqCgPaU0EbSplLC,55.149983,19,Phil Rxcket
...,...,...,...,...
149,spotify:track:7egdydsliuzDFWORdBiKMr,0.274617,Perfekt Für Dich,Tom Twers
56,spotify:track:38o5lj4mbLK34vQkJUlMrg,0.268583,London,Mokita
42,spotify:track:1xobwVandzPpC9kqAh3YAX,0.254433,Nie Genug,Fabian Wegerer
97,spotify:track:561FZrQkT2pVKtoLkz5JfE,0.254167,stranger,Drod


In [260]:
disc_02 = get_new_discoveries(df_2024_02, 2024, 2)
disc_02

,spotify_track_uri,total_minutes,track,artist
80,spotify:track:36FZL9SzRh5BhtG1cUyGWr,110.297283,Too Much,girl in red
94,spotify:track:3xJiVml80WEbicBuukBKD6,59.044017,Bisschen Liebe,LUANA
163,spotify:track:7uDUb37h7Xdhza1eWMkoJv,52.379033,Friends,Chase Atlantic
51,spotify:track:1xQnEZSuYzP9LTooTD4gIK,49.496517,Angel Numbers Amapiano (Remix),Precious
138,spotify:track:6bIJHwyaNb1txgOKbYh0rH,29.019783,Your Bones,Chelsea Cutler
...,...,...,...,...
165,spotify:track:7yNf9YjeO5JXUE3JEBgnYc,0.379783,Babydoll,Dominic Fike
39,spotify:track:1ZCLZe8hSuZl9wxuK3NiZY,0.354350,Cityljus (feat. Yasin & Blen),Petter
62,spotify:track:2TurwxJMrNgEpjWlNhVJJO,0.323650,Really Am,SK8
3,spotify:track:0K6QpySe2WtrAR2wXKheVh,0.286950,Erkläre mir die Liebe,Juju


In [262]:
disc_03 = get_new_discoveries(df_2024_03, 2024, 3)
disc_03

,spotify_track_uri,total_minutes,track,artist
288,spotify:track:6p9vTtbwiVHMahR5UwNI7k,181.948483,Ungrateful Eyes,Jon Bellion
291,spotify:track:6sMEb3eNrQiW2mxrvURMGc,160.934133,Doing Better,FLETCHER
15,spotify:track:0WmH9fXWCQ51RQi1kWPJrI,156.891733,100 Bad Days,AJR
116,spotify:track:2iv9L7pBxfuGIRbJx7ip1D,153.743717,Make Me Cry (feat. Deadbeat Girl),Ari Abdul
261,spotify:track:6EbVuwU7EnKeF2hdpViXCH,137.676450,Die Young (feat. 347aidan),Sleepy Hallow
...,...,...,...,...
219,spotify:track:58VF5ob7qRB3yUzOYEAhyf,0.228067,Lost The Breakup,Maisie Peters
296,spotify:track:6zVqpoeUx7ipuOvW97AX4w,0.215783,D.O.N.T.,Lucky Luke
265,spotify:track:6Iqz3QB6zSdn1N61acinau,0.215217,My G (feat. Ed Sheeran),Aitch
193,spotify:track:4LJCYhAAfxGM9SRQjnF5Px,0.203817,Convenient,LØLØ


In [264]:
disc_04 = get_new_discoveries(df_2024_04, 2024, 4)
disc_04

,spotify_track_uri,total_minutes,track,artist
142,spotify:track:7dIi6DS9RA6PDTU90CcNoV,172.793600,fuß baumeln,Zartmann
73,spotify:track:3VTSOMZBBbxKcSQHzQ8hys,80.434967,was wäre wenn,LUNA
102,spotify:track:51eSHglvG1RJXtL3qI5trr,72.847450,Slow It Down,Benson Boone
15,spotify:track:0tduSYti0mY7p8j5AIO4Pg,64.433933,I'm Back,girl in red
9,spotify:track:0dLAU5OcpbCefEgVPHmJfX,47.547767,If I Die,idkjack
...,...,...,...,...
36,spotify:track:1WLpHzABY3TFLkOQoU4bDW,0.242150,Mistake,Mimi Webb
21,spotify:track:144JlWqlpgkmLuRU0PvRQt,0.219067,MATCH MADE IN HELL,Dutch Melrose
61,spotify:track:2lSuT1ioDLITHNbLFi9DUa,0.218133,BURNED,Britton
25,spotify:track:1ACA277B6f46DYCgZW8di3,0.211633,if u think i'm pretty,Artemas


In [266]:
disc_05 = get_new_discoveries(df_2024_05, 2024, 5)
disc_05

,spotify_track_uri,total_minutes,track,artist
347,spotify:track:73c7iqH4lCVqu4tm66i0tY,177.449200,Risk,Gracie Abrams
25,spotify:track:0WbMK4wrZ1wFSty9F7FCgu,152.771117,"Good Luck, Babe!",Chappell Roan
181,spotify:track:3qIAER5BgecbZHfZmj7RBq,127.054300,Toast 🍸,Peter Fox
74,spotify:track:1TourCzIYFkCAkHqvslSor,93.449667,Bestie,Rosemary Joaquin
16,spotify:track:0L5XzaKc4J86e4iI9BvtIu,82.476883,Oxytocin,Chandler Leighton
...,...,...,...,...
387,spotify:track:7z6VIq5NfCmlO5GoJfMqYm,0.324700,Touch Of Summer - Radio Version,Humphrey Robertson
331,spotify:track:6habFhsOp2NvshLv26DqMb,0.254067,Despacito,Luis Fonsi
346,spotify:track:7221xIgOnuakPdLqT0F3nP,0.242633,I Had Some Help (Feat. Morgan Wallen),Post Malone
152,spotify:track:3B7cm8w1IAfOfgQHFvAsft,0.228700,Keep It Up,Good Neighbours


In [268]:
disc_06 = get_new_discoveries(df_2024_06, 2024, 6)
disc_06

,spotify_track_uri,total_minutes,track,artist
163,spotify:track:3tTuteumUYWfrwyYRlJ8WQ,67.982017,emotional capacity,Sarah Thompson
5,spotify:track:0Be9c5fb3llWN217jrMiWR,56.000000,like a girl,REYA
128,spotify:track:31QOLUDQM4LKB7Sg2aOEMF,53.191617,I'm Yours,Stephen Dawes
277,spotify:track:6f2oVJe7y0w1GO7TDxjcIy,51.284400,Serious,NERIAH
11,spotify:track:0GbSM08heTzfWO39fGKVdo,46.178367,Hot Killer,Julia Wolf
...,...,...,...,...
121,spotify:track:2sjauqCeoeFJzi27Pgrogb,0.287100,It Takes Two,Zac Efron
67,spotify:track:1eH8ZEe3AxjW3JvXOWDmPD,0.272450,Leonie,Alicia Edelweiss
282,spotify:track:6q5h5iQfAf6WIhm2ARlawe,0.258433,Röda tråden,Ella Rammelt
59,spotify:track:1WwkTtwhybWDkHEoQq1QAO,0.255467,You're Somebody Else - Acoustic,flora cash


In [270]:
disc_07 = get_new_discoveries(df_2024_07, 2024, 7)
disc_07

,spotify_track_uri,total_minutes,track,artist
141,spotify:track:6azR5gSOWDFB6pDVYMndYv,75.260300,Up At Night,Stephen Dawes
56,spotify:track:32Wy13eItx5ym93GYRpXWt,75.250417,Zzz,EDEN
157,spotify:track:7z3PblAN3dH1JMewiRydkZ,67.268067,GIRLS,The Kid LAROI
124,spotify:track:5zuPzhnGl0DRRWmquMi2MP,44.512617,Man In Finance (G6 Trust Fund),Girl On Couch
51,spotify:track:2UYNxBhod7KpjKEFlmHm2b,20.415267,Kiss Me,King Henry
...,...,...,...,...
32,spotify:track:1cXo8i9dgutdqjhjz3Zwqq,0.253050,eehhhyyy,Zartmann
145,spotify:track:6mmGDNnmFzeyGGCKAZMib5,0.245933,Pfefferkörner - 2k24 Partymix,Kreisligalegende
5,spotify:track:0AlPTC3ChJYKnBLQY11uP6,0.242017,Someone To Stay (Acoustic),Vancouver Sleep Clinic
11,spotify:track:0PfSGNUqW3h5vtJTTPohTc,0.239717,I WILL MEET YOU AT THE GRAVEYARD - TECHNO,TEKNODYN


In [272]:
disc_08 = get_new_discoveries(df_2024_08, 2024, 8)
disc_08

,spotify_track_uri,total_minutes,track,artist
295,spotify:track:7v1TTGF8kd4Mn8XssZY7rT,78.712983,covering faces,Carlie Hanson
133,spotify:track:3WOhcATHxK2SLNeP5W3v1v,72.766250,Guess featuring Billie Eilish,Charli xcx
57,spotify:track:1mcP89qPHL3qzIPoVMN8rk,71.619317,whore,mehro
185,spotify:track:5EiDSnRLeLu7jOAaBTTMdW,60.115833,Sympathy is a knife,Charli xcx
208,spotify:track:5h68SoVFGleijCtjEja3xG,55.563200,365,Charli xcx
...,...,...,...,...
281,spotify:track:7KfDVIj39efVUY57fzZm5b,0.370317,Everything is romantic,Charli xcx
94,spotify:track:2cdCC1aCXbvz9tXFOR6CxR,0.347383,Numb,Elderbrook
177,spotify:track:4xhsWYTOGcal8zt0J161CU,0.215833,Lovin On Me,Jack Harlow
96,spotify:track:2eGhEN1urbrvg0gKVOFyOm,0.213667,Out of Luck,Kevin Chung


In [274]:
disc_09 = get_new_discoveries(df_2024_09, 2024, 9)
disc_09

,spotify_track_uri,total_minutes,track,artist
189,spotify:track:7ifUSggXXVjjwJkFHcfRrX,89.601650,Girlfriend,Nieve Ella
1,spotify:track:0B4gBXd4XLXGvaBl7pI2vL,77.173783,so k.o. (feat. JEREMIAS),Levin Liam
19,spotify:track:0j2l3sf15o9vczIATwA3Xr,60.430500,black & white,Jeremy Zucker
88,spotify:track:2zjPfu8RT2BPZSOsVBor4M,49.944967,His Sofa,Nieve Ella
4,spotify:track:0EckIVFCMHtta9OiXkPouz,45.546483,Car Park,Nieve Ella
...,...,...,...,...
137,spotify:track:5PXdLJJWQJTOpfHdPkMsOw,0.330200,wannabe,renforshort
57,spotify:track:1h54366McXSNcici2LAiZP,0.301933,Smells Like Teen Spirit,Naeleck
126,spotify:track:56oGoEjA9eTZYgsttEFKY3,0.269883,The Thrill,Wiz Khalifa
0,spotify:track:08RtIVSkaKZWvFaGJqKb9o,0.250317,mir geht es gut,Levin Liam


In [276]:
disc_10 = get_new_discoveries(df_2024_10, 2024, 10)
disc_10

,spotify_track_uri,total_minutes,track,artist
61,spotify:track:2262bWmqomIaJXwCRHr13j,183.927933,Sailor Song,Gigi Perez
119,spotify:track:46sYl0o21xmXFK1a3y94Ha,138.144433,When She Smiles,Gigi Perez
75,spotify:track:2ZFryMSCFMxxVa8nY6LZ9C,92.133283,dead to me,Sam Tompkins
155,spotify:track:57l0zgPkDayvGF2NSYq0ms,74.996567,The Getaway,The Strike
99,spotify:track:3dDy7ZJZ9X2O4j5VkgCUtJ,63.681767,Nail In The Coffin,Rosie Darling
...,...,...,...,...
165,spotify:track:5PEleSkK4p4E1sx3x7cOLt,0.264567,I Just Had Sex,The Lonely Island
140,spotify:track:4ixh2ds0j86ydQvP0RjwIV,0.260450,"The Sleeping Beauty, Op. 66, TH 13 / Act 1: 6....",Pyotr Ilyich Tchaikovsky
58,spotify:track:1u6LOWtdWA2wwa3CryIBsS,0.247583,Another You,Armin van Buuren
169,spotify:track:5UZA39t4lX42ApegVubl7f,0.239667,Won’t Let You Go,Martin Garrix


In [278]:
disc_11 = get_new_discoveries(df_2024_11, 2024, 11)
disc_11

,spotify_track_uri,total_minutes,track,artist
101,spotify:track:3GYlZ7tbxLOxe6ewMNVTkw,91.416500,TV,Billie Eilish
125,spotify:track:4B0cJGASxVICLW2AsBZhiE,85.990800,LABOUR - the cacophony,Paris Paloma
42,spotify:track:17mTYn5eDcBf54CMqwloos,80.915883,Men On The Moon,Chelsea Cutler
79,spotify:track:2TwfqpwieYnr3B2xKCxTQB,52.447200,Ma Chérie - DJ Antoine & Mad Mark 2K12 Radio Edit,DJ Antoine
121,spotify:track:45kbpUIFmkS9Xla7hMXpzn,43.839633,One More Night,Maroon 5
...,...,...,...,...
109,spotify:track:3aAPTR8WFQBtAXyVkzr3Bg,0.243750,VIBE,RYSO
217,spotify:track:7mazffu6nlIv0rtRyPDMTD,0.235983,Find You,Zedd
144,spotify:track:57ngnKa3wXcN9j3rGRY2z2,0.210667,Kids of the Last Days,The Strike
124,spotify:track:49z1D7SRFdS64UGaTNdAzW,0.205783,Paper Loving,Christopher Martin


In [280]:
disc_12 = get_new_discoveries(df_2024_12, 2024, 12)
disc_12

,spotify_track_uri,total_minutes,track,artist
3,spotify:track:0zmhbBZjEEMxZLac83gLRQ,3.554433,small,Lauren Spencer Smith
2,spotify:track:0jCeblxsZemjNoHAvYD5iU,3.474600,Days On End,Brenn!
17,spotify:track:7u6FErol8bEBXOJiqBiayo,3.375783,Pretty Face,Ally Salort
12,spotify:track:6GZVFBsEiWVxWS98yU35F6,3.362433,PINS AND NEEDLES,Nessa Barrett
13,spotify:track:6PU7QGEs8f4VECOUV0ZPrX,3.236650,That’s So True - Live From Radio City Music Hall,Gracie Abrams
7,spotify:track:2XxpxQ1PeQhPMVikjKnmJy,2.936300,Before You Leave Me,Alex Warren
16,spotify:track:726SEjbF3ToEbYbn43gnxl,2.933333,Nice To Meet You,Myles Smith
15,spotify:track:6fwXcoCCisSb2NzIPKH8GI,2.892217,Joanna,Rosie Darling
9,spotify:track:4sfU6kHncVWDHM4oSdhcd1,2.888217,was wäre wenn,LUNA
10,spotify:track:4zECUk7oFraVwDUf3x6uR5,2.593967,gewinner_v2,TJARK


In [282]:
tracks_jan = get_track_stats(df_2024_01)
tracks_jan

,spotify_track_uri,track,artist,minutes_played,play_count
359,spotify:track:6KRiMyT78RNeatgjnI7sPM,немерено,лампабикт,171.239967,50
14,spotify:track:0DNAisWcJwei0pu3t8TyMM,arschloch,LUNA,93.645983,30
354,spotify:track:6GFrELGVKBXRcrTNkwjSfF,Body And Mind,girl in red,91.115033,31
375,spotify:track:6cbfq9a52AbM9OSusAJlxL,I wish you'd never,Izzy Strigel,72.594117,27
42,spotify:track:0rFAE6PNRKkw4b6nxCK7b1,THRILLER (with Clara Mae),DREAMDNVR,69.791450,28
...,...,...,...,...,...
181,spotify:track:3PgxWUQc4HJoQMj5Kp5Mcd,"Lover - Recorded at Air Studios, London",Niall Horan,0.246583,1
25,spotify:track:0TzeZppPwhEhySYaNRulr9,please,Chelsea Cutler,0.225383,1
84,spotify:track:1WqLtczgKdbHFHhefOU0y7,Self,Khalid,0.220133,1
369,spotify:track:6VrLYoQKdhu1Jruei06t65,Somebody To You,The Vamps,0.211850,1


In [284]:
tracks_feb = get_track_stats(df_2024_02)
tracks_feb

,spotify_track_uri,track,artist,minutes_played,play_count
204,spotify:track:36FZL9SzRh5BhtG1cUyGWr,Too Much,girl in red,110.297283,39
172,spotify:track:2gIko4hI8B2fepsOCo2yBb,Come Marry Me (feat. Pete Fox) - Radio Edit,Miss Platnum,93.855867,28
11,spotify:track:0DNAisWcJwei0pu3t8TyMM,arschloch,LUNA,68.747683,23
441,spotify:track:7mno5gzdkXvOpgmJWNcvxy,Toscana Fanboys (feat. Adriano Celentano),Peter Fox,61.535783,25
245,spotify:track:3xJiVml80WEbicBuukBKD6,Bisschen Liebe,LUANA,59.044017,23
...,...,...,...,...,...
85,spotify:track:1MQLdI6RONvfjEz4K6ZRXp,War,Chance Peña,0.242733,1
332,spotify:track:5g6FCPU1r3C88vUsFtat16,Don't Say You Love Me,Fifth Harmony,0.238400,1
289,spotify:track:4mFcwFOlSCxp6hjemRXwpd,hurt my feelings,Tate McRae,0.209133,1
81,spotify:track:1Ia95g66IhlAJ3qVI4qI8W,Slow,shy martin,0.208617,1


In [286]:
album_jan = get_album_stats(df_2024_01)
album_jan

,master_metadata_album_album_name,minutes_played,artist
301,verlierer,302.345050,LUNA
310,добраться до него,171.785817,лампабикт
281,if i could make it go quiet,169.595883,girl in red
212,THINK LATER,151.334483,Tate McRae
308,~how i'm feeling~,110.527867,Lauv
...,...,...,...
117,London,0.268583,Mokita
140,Nie Genug,0.254433,Fabian Wegerer
293,stranger,0.254167,Drod
130,Meet The Vamps,0.211850,The Vamps


In [102]:
# get the top genre by artist (optionally only from the top 10 artists to not wait because of rate limits)
def get_top_genre(df):
    top10 = df
    top10['genre'] = top10['master_metadata_album_artist_name'].apply(get_artist_genre)

    top10['genre'] = top10['genre'].apply(lambda x: x if isinstance(x, list) else [x])
    df_exploded = top10.explode('genre')

    genre_minutes = df_exploded.groupby('genre')['minutes_played'].sum().reset_index()

    genre_minutes = genre_minutes.sort_values(by='minutes_played', ascending=False)

    return genre_minutes

In [141]:
artist_jan = get_artist_stats(df_2024_01)
artist_jan

,master_metadata_album_artist_name,minutes_played
0,LUNA,338.662333
1,FLETCHER,291.500917
2,girl in red,281.042067
3,лампабикт,175.358083
4,Tate McRae,159.775100
...,...,...
196,Schur,0.285033
197,Tom Twers,0.274617
198,Drod,0.254167
199,Niall Horan,0.246583


In [143]:
genre_jan = get_top_genre(artist_jan)
genre_jan

,genre,minutes_played
74,pop,873.660933
0,alt z,716.656967
52,indie pop,390.054267
73,polish pop,338.662333
7,bedroom pop,281.371433
...,...,...
26,dutch house,0.313950
94,sad lo-fi,0.312267
95,sad rap,0.312267
4,australian alternative pop,0.306667


In [144]:
artist_feb = get_artist_stats(df_2024_02)
artist_feb
genre_feb = get_top_genre(artist_feb)
genre_feb

,genre,minutes_played
53,german pop,750.212233
54,german rock,415.543183
105,pop,404.275833
104,polish pop,372.513533
3,alt z,303.988750
...,...,...
89,neo-classical,0.443233
134,swedish hip hop,0.354350
136,swedish pop rap,0.354350
29,classic swedish pop,0.354350


In [288]:
artist_mar = get_artist_stats(df_2024_03)
genre_mar = get_top_genre(artist_mar)
genre_mar

,genre,minutes_played
6,alt z,1405.466900
224,pop,1350.594733
257,singer-songwriter pop,443.613067
234,pov: indie,302.528583
70,dark r&b,219.672917
...,...,...
303,trap funk,0.274633
302,trap brasileiro,0.274633
280,swedish eurodance,0.251383
167,lithuanian electronic,0.215783


In [289]:
artist_apr = get_artist_stats(df_2024_04)
genre_apr = get_top_genre(artist_apr)
genre_apr

,genre,minutes_played
3,alt z,1481.253717
117,pop,1344.596933
150,singer-songwriter pop,357.855233
126,pov: indie,228.543433
63,german pop,151.749583
...,...,...
95,mellow gold,0.295367
52,folk rock,0.295367
51,folk,0.295367
149,singer-songwriter,0.295367


In [290]:
artist_may = get_artist_stats(df_2024_05)
genre_may = get_top_genre(artist_may)
genre_may

,genre,minutes_played
4,alt z,1402.860150
132,pop,1208.195333
216,writing,779.761933
77,hoerspiel,753.475650
95,kleine hoerspiel,751.599450
...,...,...
212,vincy soca,0.615617
96,konsrock,0.584500
6,alternative pop rock,0.566883
85,indie viet,0.463500


In [291]:
artist_jun = get_artist_stats(df_2024_06)
genre_jun = get_top_genre(artist_jun)
genre_jun

,genre,minutes_played
204,pop,1723.325183
4,alt z,1138.085617
9,art pop,357.020333
242,singer-songwriter pop,346.057783
216,pov: indie,308.186667
...,...,...
201,plugg,0.417000
202,pluggnb,0.417000
223,rage rap,0.417000
171,modern indie pop,0.386783


In [292]:
artist_jul = get_artist_stats(df_2024_07)
genre_jul = get_top_genre(artist_jul)
genre_jul

,genre,minutes_played
1,alt z,1136.696383
119,pop,1102.720600
6,art pop,363.791350
146,singer-songwriter pop,309.292150
82,indie poptimism,205.453017
...,...,...
42,drill espanol,0.281367
177,urbano espanol,0.281367
0,albanian hip hop,0.254167
117,partyschlager,0.245933


In [293]:
artist_aug = get_artist_stats(df_2024_08)
genre_aug = get_top_genre(artist_aug)
genre_aug

,genre,minutes_played
171,pop,847.482500
6,art pop,402.588933
2,alt z,389.208517
196,singer-songwriter pop,379.783400
137,metropopolis,337.559167
...,...,...
225,symfonicky orchestr,0.722250
164,orchestra,0.722250
12,australian hip hop,0.446817
128,kentucky hip hop,0.215833


In [294]:
artist_sep = get_artist_stats(df_2024_09)
genre_sep = get_top_genre(artist_sep)
genre_sep

,genre,minutes_played
1,alt z,1126.411083
120,pop,1068.228683
73,indie pop,325.813650
48,electropop,279.832400
140,singer-songwriter pop,253.369350
...,...,...
72,indie folk,1.000217
168,tropical house,0.854683
15,belgian edm,0.854683
56,german indie folk,0.450900


In [295]:
artist_oct = get_artist_stats(df_2024_10)
genre_oct = get_top_genre(artist_oct)
genre_oct

,genre,minutes_played
104,pop,1239.683000
0,alt z,1147.563567
8,bedroom pop,918.585250
112,pov: indie,531.939167
71,indie pop,463.447133
...,...,...
25,comic,0.264567
24,comedy rap,0.264567
120,russian romanticism,0.260450
81,late romantic era,0.260450


In [296]:
artist_nov = get_artist_stats(df_2024_11)
genre_nov = get_top_genre(artist_nov)
genre_nov

,genre,minutes_played
148,pop,2131.682767
1,alt z,1356.699467
40,dance pop,721.495500
61,electropop,457.004333
105,indie poptimism,366.757317
...,...,...
156,pop urbaine,0.328350
14,basshall,0.328350
99,indiana hip hop,0.246683
37,complextro,0.235983


In [104]:
artist_dec = get_artist_stats(df_2024_12)
genre_dec = get_top_genre(artist_dec)
genre_dec

,genre,minutes_played
0,alt z,145.151667
51,pop,102.892400
69,swedish hip hop,40.309217
60,singer-songwriter pop,33.122917
71,swedish pop,31.673867
...,...,...
45,neo-synthpop,0.219983
36,indietronica,0.219983
16,dance rock,0.219983
5,australian electropop,0.219983


In [108]:
genre_2024 = get_top_genre(artist_stats_2024.head(1000))

C:\Users\leham\AppData\Local\Temp\ipykernel_39776\1076863147.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top10['genre'] = top10['master_metadata_album_artist_name'].apply(get_artist_genre)
C:\Users\leham\AppData\Local\Temp\ipykernel_39776\1076863147.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top10['genre'] = top10['genre'].apply(lambda x: x if isinstance(x, list) else [x])


In [109]:
genre_2024

,genre,minutes_played
353,pop,13390.372233
7,alt z,11651.866617
367,pov: indie,3405.828400
412,singer-songwriter pop,3328.200867
44,bedroom pop,2846.256133
...,...,...
50,belgian rock,3.560667
165,finnish hip hop,3.543550
164,finnish dance pop,3.543550
429,suomi rock,3.543550
